# Named entity recognition models without labelled data: A weak supervision approach to legal text

The purpose of this work is to boostrap high-quality NER models when we do not have access to training data for the target domain. See the paper for the theoretical details and related work.

## Before you start:

You should first make sure that the following Python packages are installed:
- `spacy` (version >= 2.2)
- `hmmlearn`
- `snips-nlu-parsers`
- `pandas`
- `numba`

You should also install the `en_core_web_sm` and `en_core_web_md` models in Spacy.

To run the neural models in `ner.py`, you need also need `pytorch`, `cupy`, `keras` and `tensorflow` installed. 

To run the baselines, you will also need to have `snorkel` installed.

Finally, you also need to download the following files and add them to the `data` directory:
- [`conll2003_spacy.tar.gz`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/conll2003_spacy.tar.gz) (unpack the archive in the same directory)
- [`BTC_spacy.tar.gz`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/SEC_spacy.tar.gz) (same)
- [`SEC_spacy.tar.gz`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/SEC_spacy.tar.gz) (same)
- [`wikidata.json`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/wikidata.json)
- [`wikidata_small.json`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/wikidata_small.json)
- [`crunchbase.json`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/crunchbase.json)
- [`conll2003.docbin`](https://github.com/anonymous-NLP/weak-supervision-for-NER/releases/download/acl2020/conll2003.docbin)
   
    

## Introduction

In [1]:
from os import path
with open(path.join("test", "text.txt"), "rb") as f:
    news_text = f.read().decode()

import re
# news_text = re.sub('\s+', ' ', news_text)

To get things started, let's look at the named entities recognised by a standard NER model (from Spacy):

In [2]:
%load_ext autoreload
%autoreload 2
import annotations

In [3]:
import spacy

# We load the spacy model
nlp = spacy.load("es_core_news_sm", )
doc = nlp(news_text)

<br>
As we can see from the results above, the named entity recognition contains quite a lot of errors. Atlanta is strangely labelled as an organisation, while "Best Buy" is ignored at several places in the document.  IPhone is also either labelled as an organisation or even as a location.

A slightly larger neural model (again from Spacy) works better, but still contains quite a few errors and omissions:

Ideally, one would wish to train a better named entity recognition model, which is better tailored to the specific needs and linguistic patterns found in these articles. However, although raw text data is not difficult to acquire, we often do not have access to labelled data. To address this issue, we developed an alternative approach based on __weak supervision__, combining several (noisy) supervision sources instead of relying on a single "gold standard". Indeed, we do have access to several possible supervision sources, such as alternative NER models trained on other corpora, large lists of entity (companies, person names, geographical locations), shallow linguistic patterns, and document-level constraints. 

The key idea behind the proposed approach is thus to (1) use these supervision sources to automatically annotate news corpora, (2) estimate a label model (more precisely an HMM model) that unifies all these sources into a single one, and (3) learn a new NER model based on these unified labels. <br>

__Outline of this notebook__: We describe below the various labelling functions.  We then explain how these various sources can be merged into a single source. Finally, we detail the architecture behind the NER model.

## __Step 1:__ Labelling functions

### 1) Other data-driven NER models

A first source of automatic annotation comes from NER models trained on multiple, distinct corpora. We went through [available NE-labelled corpora](https://github.com/juand-r/entity-recognition-datasets) to search for datasets that could be used to train alternative models. We then trained Spacy models for all of them, and then conducted some experiments to assess their performance. At the end of the process, we ended up with four models:
- The standard Spacy model for English (`en_core_web_md`), trained on Ontonotes v5
- A model trained on [ConLL 2003](https://www.clips.uantwerpen.be/conll2003/ner/)
- A model trained on the [Broad Twitter Corpus](https://github.com/GateNLP/broad_twitter_corpus)
- A model trained on a corpus of [SEC filings](https://www.aclweb.org/anthology/U15-1010/).
    
Note there are differences between the entity labels of these models: while Ontonotes contains no less than [18 classes](https://spacy.io/api/annotation#named-entities), the other corpora only contain `PER(SON)`, `ORG`, `LOC` and `MISC`. Furthermore, the labels also do not match each other perfectly: while Ontonotes distinguishes between geopolitical locations (`GPE`) and "natural" locations (such as continents, seas etc., labelled as `LOC`), the three other models regroup all geographical entities as `LOC`. 

We can apply annotations from a Spacy model using the `ModelAnnotator` class

As we can see, the results are not perfect on this model either, but the errors are distinct from the ones made by the Ontonotes model. 

The annotations are written in the `user_data` dictionary of the Spacy document:

Each `ModelAnnotator` adds two annotation sources: one that is directly based on the Spacy Model (here `conll2003`), and one that also includes the corrections specified in the method `_correct_entities` (in `spacy_wrapper.py`) that we implemented earlier this year.  The corrected version are indicated with a `+c` suffix.

Here are the results from the three other models:

__Note__: When annotating large collections of news documents, the method `annotator.pipe(news_docs)` is much more efficient than calling `annotate(...)` every single time, as it batches the documents on which to run the NER model.

### 2) Gazetteers

Another useful source of annotation comes from large lists of entities such as persons, places and organisations. The gazetteers are using a _trie_ to efficiently search for occurrences in the text. Each gazetteer creates two annotation sources: one that is case-sensitive (`_cased` suffix) and one case-insentitive (`_uncased` suffix).

Defendants Judgements of Constitucional court of Colombia

In [4]:
import labelling 

In [9]:
def init_annotator(doc):
    annotator_city = annotations.GazetteerAnnotator('./data/city.json', "city")
    annotator_big_city = annotations.GazetteerAnnotator('./data/big_city.json', "big_city")
    annotator_country = annotations.GazetteerAnnotator('./data/country.json', "country")
    annotator_municipality = annotations.GazetteerAnnotator('./data/municipality.json', "municipality")
    annotator_gov = annotations.GazetteerAnnotator('./data/government_agency.json', "government_agency")

    full_annotator = annotations.FullAnnotator()
    doc.user_data["annotations"] = {}
    full_annotator.add_annotator(annotator_municipality)
    full_annotator.add_annotator(annotator_city)
    full_annotator.add_annotator(annotator_big_city)
    full_annotator.add_annotator(annotator_country)
    full_annotator.add_annotator(annotator_gov)

    for source_name in ["municipality", "city","country", "government_agency"]:
        for cased in ["cased", "uncased"]:
            full_annotator.add_annotator(annotations.SpanConstraintAnnotator(lambda s: len(
                s) > 1, "%s_%s" % (source_name, cased), "multitoken_"))


    annotator_law = annotations.FunctionAnnotator(annotations.law_rules, "law_rules")
    full_annotator.add_annotator(annotator_law)

    annotator_particpant= annotations.FunctionAnnotator(annotations.particpant_rules, "participants")
    full_annotator.add_annotator(annotator_particpant)

    annotator_magister= annotations.FunctionAnnotator(annotations.magister_rules, "magister")
    full_annotator.add_annotator(annotator_magister)


    annotator_judge= annotations.FunctionAnnotator(annotations.judge_rules, "judge")
    full_annotator.add_annotator(annotator_judge)

    annotator_chamber= annotations.FunctionAnnotator(annotations.chamber_rules, "chamber")
    full_annotator.add_annotator(annotator_chamber)

    annotator_judgement= annotations.FunctionAnnotator(annotations.judgement_rules, "judgement")
    full_annotator.add_annotator(annotator_judgement)

    annotator_right= annotations.FunctionAnnotator(annotations.right_rules, "right")
    full_annotator.add_annotator(annotator_right)

    annotator_decret = annotations.FunctionAnnotator(annotations.decret_rules, "decret")
    full_annotator.add_annotator(annotator_decret)


    annotator_doc = annotations.FunctionAnnotator(annotations.doc_rules, "document")
    full_annotator.add_annotator(annotator_doc)

    annotator_resolution = annotations.FunctionAnnotator(annotations.resolution_rules, "resolution")
    full_annotator.add_annotator(annotator_resolution)

    annotator_court= annotations.FunctionAnnotator(annotations.court_rules, "court")
    full_annotator.add_annotator(annotator_court)

    annotator_m = annotations.DocumentMajorityAnnotator()
    full_annotator.annotate(doc)
    full_annotator.add_annotator(annotator_m) 
    annotator_m.annotate(doc)
    
    return full_annotator

def annotation_all(doc, display=False):

    #annotator_m = annotations.DocumentMajorityAnnotator()
    #full_annotator.annotate(doc)
    #full_annotator.add_annotator(annotator_m)

    full_annotator = init_annotator(doc)
    annotations.display_entities(doc, "doc_majority_uncased", display=display)
    #annotator_m.annotate(doc)
    #full_annotator.annotate(doc)

annotation_all(doc, True)
#annotations.display_entities(doc, "doc_majority_uncased")

Extracting data from ./data/city.json
Populating trie for entity class CTY (number: 5454)
Extracting data from ./data/big_city.json
Populating trie for entity class CTY (number: 3117)
Extracting data from ./data/country.json
Populating trie for entity class COUNTRY (number: 168)
Extracting data from ./data/municipality.json
Populating trie for entity class MUN (number: 1819)
Extracting data from ./data/government_agency.json
Populating trie for entity class GOV (number: 1379)
() ********************
() ********************


In [7]:
from utils import export_iob
encoding='iso-8859-1'
export_iob(doc, "file.iob")
       

In [12]:
os.listdir("./text/")

['T-001-11.txt',
 'T-001-12.txt',
 'T-001-13.txt',
 'T-001-14.txt',
 'T-001-15.txt',
 'T-001-16.txt',
 'T-001-17.txt',
 'T-001-18.txt',
 'T-001-19.txt',
 'T-002-11.txt',
 'T-002-12.txt',
 'T-002-13.txt',
 'T-002-14.txt',
 'T-002-15.txt',
 'T-002-16.txt',
 'T-002-17.txt',
 'T-002-18.txt',
 'T-002-19.txt',
 'T-002A-17.txt',
 'T-003-11.txt',
 'T-003-12.txt',
 'T-003-13.txt',
 'T-003-14.txt',
 'T-003-15.txt',
 'T-003-16.txt',
 'T-003-18.txt',
 'T-003-19.txt',
 'T-004-11.txt',
 'T-004-12.txt',
 'T-004-13.txt',
 'T-004-14.txt',
 'T-004-15.txt',
 'T-004-16.txt',
 'T-004-18.txt',
 'T-004-19.txt',
 'T-005-11.txt',
 'T-005-12.txt',
 'T-005-13.txt',
 'T-005-14.txt',
 'T-005-15.txt',
 'T-005-16.txt',
 'T-005-18.txt',
 'T-005-19.txt',
 'T-006-11.txt',
 'T-006-13.txt',
 'T-006-14.txt',
 'T-006-15.txt',
 'T-006-18.txt',
 'T-006-19.txt',
 'T-007-11.txt',
 'T-007-13.txt',
 'T-007-14.txt',
 'T-007-15.txt',
 'T-007-19.txt',
 'T-008-11.txt',
 'T-008-12.txt',
 'T-008-13.txt',
 'T-008-14.txt',
 'T-008-15.tx

In [11]:
import os
for filaname in os.listdir("./text/")[15:]:
    name, _ = filaname.split(".")
    with open( f"./text/{filaname}", "rb") as f:
        text = f.read().decode()

    text = re.sub('\s{2,}', ' |', text)
    text = re.sub(r'\. \|', '.\n', text)
    text = re.sub(r' \|', ' ', text)
    doc = nlp(text)
    annotation_all(doc)
    export_iob(doc, f"./iob/{name}.iob")

T-001-11
Extracting data from ./data/city.json
Populating trie for entity class CTY (number: 5454)
Extracting data from ./data/big_city.json
Populating trie for entity class CTY (number: 3117)
Extracting data from ./data/country.json
Populating trie for entity class COUNTRY (number: 168)
Extracting data from ./data/municipality.json
Populating trie for entity class MUN (number: 1819)
Extracting data from ./data/government_agency.json
Populating trie for entity class GOV (number: 1379)
T-001-12
Extracting data from ./data/city.json
Populating trie for entity class CTY (number: 5454)
Extracting data from ./data/big_city.json
Populating trie for entity class CTY (number: 3117)
Extracting data from ./data/country.json
Populating trie for entity class COUNTRY (number: 168)
Extracting data from ./data/municipality.json
Populating trie for entity class MUN (number: 1819)
Extracting data from ./data/government_agency.json
Populating trie for entity class GOV (number: 1379)
T-001-13
Extracting 

AttributeError: 'NoneType' object has no attribute 'label'

In [ ]:
doc.user_data["annotations"]["doc_majority_uncased"]

In [1]:
doc.text

NameError: name 'doc' is not defined

In [ ]:
annotator = annotations.GazetteerAnnotator('./data/city.json', "city")

annotator.annotate(doc)
annotations.display_entities(doc, "city_uncased")

In [ ]:
annotator = annotations.GazetteerAnnotator('./data/plaintiff.json', "plaintiff")

annotator.annotate(doc)
annotations.display_entities(doc, "plaintiff_uncased")

In [ ]:
annotator = annotations.GazetteerAnnotator('./data/judgements.json', "judgements")

annotator.annotate(doc)
annotations.display_entities(doc, "judgements_uncased")


#### 2.1) Wikipedia
The database from Wikipedia is extracted from the [NECKar](https://event.ifi.uni-heidelberg.de/?page_id=532) dataset.  The postprocessing (which, among others, filters out entities that are also relatively common English words) is implemented in `compile_wikidata`. In addition, we also extracted from Wikidata a list of commercial products and added them to the gazetteer. 

In [ ]:
annotator = annotations.GazetteerAnnotator(annotations.WIKIDATA, "wiki")

annotator.annotate(doc)
annotations.display_entities(doc, "wiki_cased")
annotations.display_entities(doc, "wiki_uncased")

Again, the annotation model does make some errors: `Moore` is thought to be a [geopolitical entity](https://en.wikipedia.org/wiki/Moore) instead of a person. Note that `AT&T` has two alternative labels: `ORG` or `GPE` (see [AT&T station](https://en.wikipedia.org/wiki/AT%26T_(SEPTA_station))).  

In addition to the full wiki data, we also added a specific gazetteer that only employs wikidata objects containing a text description:

In [ ]:
annotator = annotations.GazetteerAnnotator(annotations.WIKIDATA_SMALL, "wiki_small")

annotator.annotate(doc)
annotations.display_entities(doc, "wiki_small_cased")
annotations.display_entities(doc, "wiki_small_uncased")

As we can see, the second gazetteer has a higher precision than the first (at a cost of lower coverage).

#### 2.2 Crunchbase

The second gazetteer is extracted from the [Open Data Map from Crunchbase](https://data.crunchbase.com/docs/open-data-map), which contains lists of both organisations and (business) persons.

In [ ]:
import annotations
annotator = annotations.GazetteerAnnotator(annotations.CRUNCHBASE, "crunchbase")

annotator.annotate(doc)
annotations.display_entities(doc, "crunchbase_cased")

#### 2.3 Geonames

The [geonames](http:www.geonames.org) database contains a large list of locations, including both geopolitical entities and "natural" locations:

In [ ]:
annotator = annotations.GazetteerAnnotator(annotations.GEONAMES, "geo")

annotator.annotate(doc)
annotations.display_entities(doc, "geo_uncased")

Note that the annotator explicitly marks the detected entities with the label `COMPANY` instead of the more generic `ORG`.  

#### 2.4 Product names

Finally, we used [DBPedia](http://www.dbpedia.org) to extract a list of products and brands, since the recognition of products is particularly poor in Spacy NER models:

In [ ]:
annotator = annotations.GazetteerAnnotator(annotations.PRODUCTS, "product")

annotator.annotate(doc)
annotations.display_entities(doc, "product_cased")

#### 2.5 Other entities

Finally, we also run a detector using handcrafted lists of countries, languages, nationalities, and religious/political groups.

In [ ]:
# Detection of misc entities
misc_annotator = annotations.FunctionAnnotator(annotations.misc_generator, "misc_detector", to_exclude=exclusives)
annotations.display_entities(doc, "misc_detector")

### 3. Shallow patterns

Some named entities can also be captured through relatively simple, handcrafted patterns defined on the Spacy document. The class `FunctionAnnotator` makes it easy to define an annotator based on a function that takes a Spacy document as input and generate text spans with a label. Relations of mutual exclusivity between annotation sources can also be specified in the annotator. For instance, we can specify that numbers that are part of a date, time or money span should be ignored from the "number_detector" (to avoid having e.g. the `21` in `October 21` labelled as a `CARDINAL`): 

In [ ]:
date_annotator = annotations.FunctionAnnotator(annotations.date_generator, "date_detector")
time_annotator = annotations.FunctionAnnotator(annotations.time_generator, "time_detector")
money_annotator = annotations.FunctionAnnotator(annotations.money_generator, "money_detector")
exclusives = ["date_detector", "time_detector", "money_detector"]
number_annotator = annotations.FunctionAnnotator(annotations.number_generator, "number_detector", exclusives)

date_annotator.annotate(doc)
time_annotator.annotate(doc)
money_annotator.annotate(doc)
number_annotator.annotate(doc)
annotations.display_entities(doc, "date_detector")
annotations.display_entities(doc, "time_detector")
annotations.display_entities(doc, "money_detector")
annotations.display_entities(doc, "number_detector")

We have also created a range of patterns aiming to improve the _detection_ of named entities, even though they leave the actual label underspecified (as a generic `ENT` label). Four such detectors are constructed:
- two detectors of proper names based on casing (marking sequence of tokens whose lemma are "titled" as potential named entities)
- one detector of NNP sequences (based on the Spacy POS tagger)
- and one detector of sequences with proper names linked with "compound" dependency relations

In [ ]:
# Detection based on casing, but allowing some lowercased tokens
proper_detector = annotations.SpanGenerator(annotations.is_likely_proper)
proper_annotator = annotations.FunctionAnnotator(proper_detector, "proper_detector",to_exclude=exclusives)

# Detection based on casing, but allowing some lowercased tokens
proper2_detector = annotations.SpanGenerator(annotations.is_likely_proper, exceptions=annotations.LOWERCASED_TOKENS)
proper2_annotator = annotations.FunctionAnnotator(proper2_detector, "proper2_detector",  to_exclude=exclusives)
        
# Detection based on part-of-speech tags
nnp_detector = annotations.SpanGenerator(lambda tok: tok.tag_=="NNP")
nnp_annotator = annotations.FunctionAnnotator(nnp_detector, "nnp_detector", to_exclude=exclusives)
        
# Detection based on dependency relations (compound phrases)
compound_detector = annotations.SpanGenerator(lambda x: annotations.is_likely_proper(x) and annotations.in_compound(x))
compound_annotator = annotations.FunctionAnnotator(compound_detector, "compound_detector", to_exclude=exclusives)

proper_annotator.annotate(doc)
proper2_annotator.annotate(doc)
nnp_annotator.annotate(doc)
compound_annotator.annotate(doc)

annotations.display_entities(doc, "proper_detector")
annotations.display_entities(doc, "proper2_detector")
annotations.display_entities(doc, "nnp_detector")
annotations.display_entities(doc, "compound_detector")

Furthermore, we created three specific annotators to recognise:
- company names with a legal type
- full person names (with a first name along a list of common first names)
- legal references of type `LAW`.

In [ ]:
# Detection of companies with legal type
company_annotator = annotations.FunctionAnnotator(annotations.CompanyTypeGenerator(),
                                                  "company_type_detector", to_exclude=exclusives)
exclusives +=["company_type_detector"]
        
# Detection of full person names
full_name_annotator = annotations.FunctionAnnotator(annotations.FullNameGenerator(), 
                                                    "full_name_detector", to_exclude=exclusives)

# Detection of legal references
legal_annotator = annotations.FunctionAnnotator(annotations.legal_generator, "legal_detector", to_exclude=exclusives)

company_annotator.annotate(doc)
full_name_annotator.annotate(doc)
misc_annotator.annotate(doc)
legal_annotator.annotate(doc)
annotations.display_entities(doc, "company_type_detector")
annotations.display_entities(doc, "full_name_detector")
annotations.display_entities(doc, "misc_detector")
annotations.display_entities(doc, "legal_detector")

Finally, we also rely on an external probabilistic [parser of named entities](https://github.com/snipsco/snips-nlu-parsers) from [Snips](https://snips.ai/). The parser recognises `DATE`, `TIME`, `ORDINAL`, `CARDINAL`, `MONEY` and `PERCENT`. The parser is implemented in _Rust_, so it runs quite fast.

### 4. Document-level annotators

All annotators presented so far rely on _local_ decisions on tokens or phrases.  However, news articles are not mere collections of words, but exhibit a high degree of internal coherence. This can be exploited to furhter improve the annotation. Two document-level annotators are implemented:

Before we can run the document-level annotators, we need to normalise some of the entities. The `StandardiseAnnotator` is responsible for this normalisation:
- entities `PER` (from conll2003, BTC and SEC) are set to `PERSON`
- entities `LOC` from conll2003, BTC and SEC for spans that are also annotated by other layers as `GPE` are set to `GPE` 
- entities `ORG` that are annotated by other layers as `COMPANY` are set to `COMPANY`
    

In [ ]:
annotator = annotations.StandardiseAnnotator()
doc = annotator.annotate(doc)

#### 4.1 Document history

When a journalist first mentions an entity such as a company or person in an article, they typically write it in a "long form", and then use shorter mentions once the entity is properly introduced. For instance, in the text above, "Scott Moore" is first mentioned with a full name, and then simply referred to as "Moore". Similarly, companies are often first introduced to with their legal type.  The `DocumentHistoryAnnotator` takes advantage of this property, by propagating the label from the first mention onto subsequent mentions:

In [ ]:
annotator = annotations.DocumentHistoryAnnotator()
annotator.annotate(doc)
annotations.display_entities(doc, "doc_history")

#### 4.2 Label consistency

Another property of news documents is the fact that two (or more) named entities sharing the same string in a text typically refer to the same entity, and should therefore have the same label. "Komatsu" can be both a company name and a city in Japan, but within a given document, it will typically be one or the other for the whole document. We can capture this fact with an annotator that looks at the majority label for a given string, and annotate all occurrences with this label:

In [ ]:
annotator = annotations.DocumentMajorityAnnotator()
annotator.annotate(doc)
annotations.display_entities(doc, "doc_majority_cased")

## __Step 2__: Estimation of label model

We can construct a full annotator with all annotators described above, and then run it on a dataset from the target domain:

In [ ]:
import annotations
full_annotator = annotations.FullAnnotator().add_all()
print("Total number of annotators:", len(full_annotator.annotators))

We can then take the raw data from CoNLL 2003, run Spacy on the textual content, and finally apply the annotator to get annotations from the each source:

In [ ]:
# We annotate the documents and store them in a Spacy DocBin fileb
full_annotator.annotate_docbin("./data/conll2003.docbin")

One this is done, we can finally estimate a unified annotator model through weak supervision. The basic idea is to describe the named entity recognition problem as a _Hidden markov Model_ where the observations are the annotations from each source, and the states correspond to the "true" (hidden) labels for each token, as illustrated below.

<img src="data/hmm.png">

Since we don't have access to the true labels for each token, we will rely on _Baum-Welch_ (a variant of EM) to estimate the HMM model through unsupervised training. More specifically, we will need to estimate 3 models:
- the initial probabilities $P(Y_0)$ of the labels for the first token of a document
- the transition matrix $P(Y_i | Y_{i-1})$ for the labels 
- the emission models $P(\lambda_{i,j} | Y_i)$ of observing a particular value $\lambda_{i,j}$ (say, `B-PER`) from the source $j$ given the true label $Y_i$. In the current model, we assume the emissions to be independent of one another given the true label, to reduce the complexity of the model.

Given an annotated dataset, the HMM model can be easily estimated:

In [ ]:
import labelling 

# We create the unified model (and make sure the CoNLL 2003-trained NER model is ignored)
sources_to_use = [l for l in labelling.SOURCE_NAMES if "conll2003" not in l]
unified_model = labelling.HMMAnnotator(sources_to_use)

# We then run Baum-Welch on the model (can take some time)
unified_model.train("./data/conll2003.docbin")

# Saving the model to a file
unified_model.save("./data/hmm_conll2003.pkl")

Note that the HMM model relies on some informative priors to facilitate the parameter estimation:
- the prior for the initial probabilities is a Dirichlet based on counts for the most reliable model
- the prior for the transition matrix is a list of Dirichlet also based on counts from the standard Spacy NER model.
- finally, the initial emission models are calculated based on subjective estimates of the relative precision and recall of each source. For instance, we know that a source like `company_type_detector` (which looks at legal suffixes such as "Inc." at the end of the noun phrase) has a very high precision, but a low recall , since many mentions of companies do not include a suffix. In contrast, gazeteers will tend to have a better recall, but a lower precision (some company names also happen to be names of geopolitical entities or persons).  The initial precisions and recalls provided to the model is specified in `SOURCE_PRIORS` in the file `labelling.py`. When a precision and recall is not provided for a given source, they are assumed to be zeros (for instance, `company_type_detector` only detects `COMPANY` entities and nothing else).  

One the model is learned, we can apply it as any other "annotator" object:

In [ ]:
import labelling
full_annotator.annotate(doc)
unified_model.annotate(doc)
annotations.display_entities(doc, "HMM")

And we can apply it to the full dataset:

In [ ]:
unified_model.annotate_docbin("./data/conll2003.docbin")

<br>

## __Step 3__: Development of neural NER model


We can now learn a neural NER model based on these unified annotations. We have two options: a straighforward (but slightly underperforming) approach using Spacy, and a more sophisticated approach using our own NER model

### __Alternative 1__: Using Spacy

In [ ]:
import annotations 
annotations.convert_to_json("./data/conll2003.docbin", "./data/conll2003_dev.json", cutoff=20)
annotations.convert_to_json("./data/conll2003.docbin",  "./data/conll2003_train.json", nb_to_skip=20)

# We need to convert COMPANY into ORG if we want to use the standard Spacy model as starting point
!sed -i 's/COMPANY/ORG/g' ./data/conll2003_train.json
!sed -i 's/COMPANY/ORG/g' ./data/conll2003_dev.json

And we can then directly train a new NER model with Spacy's training regime:

In [ ]:
!rm -rf ./data/conll2003_spacy
import spacy
spacy.cli.train(lang="en", output_path="./data/conll2003_spacy", 
                train_path="./data/conll2003_train.json", 
                dev_path="./data/conll2003_dev.json",
               base_model="en_core_web_md", vectors="en_core_web_md",
               pipeline="ner", n_iter=5)

We then load the learned model (taking the model version with best performance on the development set):

In [ ]:
nlp = spacy.load("./data/conll2003_spacy/model-best")

And run it on the document:

In [ ]:
annotations.display_entities(nlp(news_text))

To add the results of this neural model to the results, we can use the `ModelAnnotator` object:

In [ ]:
model = annotations.ModelAnnotator("./data/conll2003_spacy/model-best", "aggregated_spacy")
model.annotate_docbin("./data/conll2003.docbin")

### __Alternative 2__: Using custom NER model

Although the model above performs relatively well, its performance can be further improved by using more advanced neural architectures, implemented in file `ner.py`. 

In [ ]:
import ner

# We extract the training and validation documents (as Spacy docs)
val_docs = ner.generate_from_docbin("../data/conll2003.docbin", target_source="HMM", cutoff=20, loop=False) 
train_docs = ner.generate_from_docbin("../data/conll2003.docbin", target_source="HMM", nb_to_skip=20,loop=True) 

# We create the NER model
model = ner.NERModel(epoch_length=1000, dropout=0.5, batch_size=8, lr=0.01, nb_epochs=4, 
                     trainable_word_embeddings=False, word_emb_transform_dim=128)
# NB: to reduce overfitting on this toy dataset, we don't tune the word embeddings here, 
# but instead pass them through a dense layer

print("finished building model, now training...")
model.train(train_docs, val_docs)

In [ ]:
model.label(doc)
annotations.display_entities(doc)

As we can see, the model still make some errors, due to the fact that the training set (200 documents) remains too small to train a decent neural NER model. It should also be noted that the Spacy model above used the standard Spacy model for English (`en_core_web_md`) as starting point, while the custom NER model above is learned from scratch.

## __Step 4__: Evaluation

We'll first evaluate the approach on a standard dataset, namely conLL 2003. Note that ConLL only makes use of 4 labels (`PER`, `ORG`, `LOC`, `MISC`) instead of the 18 labels which were used above.

#### Ontonotes-trained NER

The simplest baseline is to use a neural NER model trained on Ontonotes. In other words, this baseline considers one single source, namely a neural model trained on Ontonotes. Fortunately, we already have such a model, namely the labelling function `core_web_md`.

#### Majority voting

Another baseline consists in using majority voting on the various sources:

In [ ]:
import labelling
mv = labelling.MajorityVoter(sources_to_use)
mv.annotate_docbin("./data/conll2003.docbin")

#### Snorkel model

Snorkel is another weak supervision framework which we use to compare our approach:

In [ ]:
snorkel_model = labelling.SnorkelModel(sources_to_use)
snorkel_model.train("./data/conll2003.docbin")
snorkel_model.annotate_docbin("./data/conll2003.docbin")

#### Mixtures of multinomials

For the mixtures of multinomials, we rely on code written in `R`, and available in the directory `mixtures`.

#### AdaptaBERT

See https://github.com/xhan77/AdaptaBERT.

### Metrics

The file `analysis.py` contains code to easily extract evaluation metrics by comparing the annotations from a particular annotation layer (for instance the HMM predictions, or the predictions from a single source) to the gold standard:

In [ ]:
import analysis, annotations

# Extractint the documents
docs = list(annotations.docbin_reader("./data/conll2003.docbin"))

# Here we extract for illustration purposes the metrics for 5 approaches: the Ontonotes-trained NER,
# the majority voter, the Snorkel model, the HMM, and the Spacy neural model traind on the aggregated sources
df = analysis.evaluate(docs, ["core_web_md", "majority_voter", "snorkel", "HMM", "aggregated_spacy"], analysis.CONLL_MAPPINGS, analysis.CONLL_TO_RETAIN)
df.loc["micro"]

Note: the results in the above table are not exactly the same as the ones reported in the original paper (the results above are actually slightly better!), due to some last-minute changes in the implementation of the labelling functions.

### Miscellaneous

The file `data/crowdsourced.docbin` contains the NER-annotated sentences annotated via crowd-sourcing.